In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle


df = pd.read_csv("facturas_procesadas.csv")

# agrupar importes por año
df_agg = df.groupby("year")["importe"].sum().reset_index()

# crear nuevas clases para mejorar la prediccion
df_agg["importe_anterior"] = df_agg["importe"].shift(1)  # Importe del año anterior
df_agg["variacion_anual"] = df_agg["importe"] - df_agg["importe_anterior"]  # Cambio de un año a otro
df_agg["tasa_crecimiento"] = df_agg["variacion_anual"] / df_agg["importe_anterior"]  # % de crecimiento
df_agg = df_agg.fillna(0)


# crear nuevas variables  X e y
X_regresion = df_agg[["year", "importe_anterior", "variacion_anual", "tasa_crecimiento"]]
y_regresion = df_agg["importe"]

# dividir en conjunto de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X_regresion, y_regresion, test_size=0.2, random_state=42)

# entrenar el modelo de regresion RandomForest
modelo_regresion = RandomForestRegressor(n_estimators=100, random_state=42)
modelo_regresion.fit(X_train, y_train)

# evaluacion del modelo
y_pred = modelo_regresion.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f" MAE: {mae:.2f}")
print(f" RMSE: {rmse:.2f}")
print(f" R² Score: {r2:.2f}")



# predecir el importe total del proximo año
proximo_anio = df_agg["year"].max() + 1
ultimo_importe = df_agg["importe"].iloc[-1]
variacion = df_agg["variacion_anual"].iloc[-1]
tasa = df_agg["tasa_crecimiento"].iloc[-1]

entrada_nueva = pd.DataFrame([[proximo_anio, ultimo_importe, variacion, tasa]],
                              columns=["year", "importe_anterior", "variacion_anual", "tasa_crecimiento"])
prediccion = modelo_regresion.predict(entrada_nueva)

print(f" Predicción de Importe Total para el año {proximo_anio}: €{prediccion[0]:,.2f}")


with open("modelo_prediccion_anual.pkl", "wb") as f:
    pickle.dump(modelo_regresion, f)




 MAE: 5301872.77
 RMSE: 5752853.80
 R² Score: 0.13
 Predicción de Importe Total para el año 2026: €13,732,951.33
